In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords

In [2]:
# Utility function to clean text.
def text_cleaner(text):
    
    # Visual inspection shows spaCy does not recognize the double dash '--'.
    # Better get rid of it now!
    text = re.sub(r'--',' ',text)
    
    # Get rid of headings in square brackets.
    text = re.sub("[\[].*?[\]]", "", text)
    
    # Get rid of chapter titles.
    text = re.sub(r'Chapter \d+','',text)
    
    # Get rid of extra whitespace.
    text = ' '.join(text.split())
    
    return text[0:900000]


# Import all the Austen in the Project Gutenberg corpus.
austen = ""
for novel in ['persuasion','emma','sense']:
    work = gutenberg.raw('austen-' + novel + '.txt')
    austen = austen + work

# Clean the data.
austen_clean = text_cleaner(austen)

In [4]:
# Parse the data. This can take some time.
nlp = spacy.load('en_core_web_sm')
austen_doc = nlp(austen_clean)

In [5]:
# Organize the parsed doc into sentences, while filtering out punctuation
# and stop words, and converting words to lower case lemmas.
sentences = []
for sentence in austen_doc.sents:
    sentence = [
        token.lemma_.lower()
        for token in sentence
        if not token.is_stop
        and not token.is_punct
    ]
    sentences.append(sentence)


print(sentences[20])
print('We have {} sentences and {} tokens.'.format(len(sentences), len(austen_clean)))

['daughter', 'eld', 'give', 'thing', 'tempt']
We have 8323 sentences and 900000 tokens.


In [6]:
import gensim
from gensim.models import word2vec

model = word2vec.Word2Vec(
    sentences,
    workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
    min_count=10,  # Minimum word count threshold.
    window=6,      # Number of words around target word to consider.
    sg=0,          # Use CBOW because our corpus is small.
    sample=1e-3 ,  # Penalize frequent words.
    size=300,      # Word vector length.
    hs=1           # Use hierarchical softmax.
)

print('done!')

done!


In [7]:
# List of words in model.
vocab = model.wv.vocab.keys()

print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
print(model.doesnt_match("breakfast marriage dinner lunch".split()))

[('musgrove', 0.8991125226020813), ('clay', 0.8775729537010193), ('benwick', 0.8763314485549927), ('goddard', 0.8737025260925293), ('harville', 0.8607302904129028), ('wentworth', 0.829587459564209), ('colonel', 0.7991381883621216), ('hall', 0.7819957733154297), ('room', 0.7712162733078003), ('join', 0.7599485516548157)]
0.8842774
marriage


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  # This is added back by InteractiveShellApp.init_path()
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/gensim/models/keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


From my understanding The only parameters to twick in oreder to tune our word2vec model are 'sg'(skip-gram)= 1 which is good when the word counts is large or 0(CBOW) otherwise.the 'size' which is the word vector length and the 'hs' which is either heirarchical as 1 and Negative sampling as 0.'sg' is oviously 0 or CBOW because of small number of words.

In [8]:
param = {'size':range(100,400,50), 'hs': [0,1]}

In [11]:
for i in param['size']:
    for j in param['hs']:
        model = word2vec.Word2Vec(
                        sentences,
                        workers=4,     # Number of threads to run in parallel (if your computer does parallel processing).
                        min_count=10,  # Minimum word count threshold.
                        window=6,      # Number of words around target word to consider.
                        sg=0,          # Use CBOW because our corpus is small.
                        sample=1e-3 ,  # Penalize frequent words.
                        size= i,      # Word vector length.
                        hs=j           # Use hierarchical softmax.
                        )
        # List of words in model.
    vocab = model.wv.vocab.keys()

    print(model.wv.most_similar(positive=['lady', 'man'], negative=['woman']))

# Similarity is calculated using the cosine, so again 1 is total
# similarity and 0 is no similarity.
    print(model.wv.similarity('mr', 'mrs'))

# One of these things is not like the other...
    print(model.doesnt_match("breakfast marriage dinner lunch".split()))

print('done!')

[('benwick', 0.8306616544723511), ('clay', 0.8245633840560913), ('musgrove', 0.8101413249969482), ('harville', 0.8091698884963989), ('colonel', 0.7965329885482788), ('goddard', 0.7832509279251099), ('excessively', 0.7648067474365234), ('wentworth', 0.755462110042572), ('wallis', 0.7505685091018677), ('croft', 0.7495462894439697)]
0.8556001
marriage
[('benwick', 0.8825939893722534), ('harville', 0.8727622628211975), ('clay', 0.8672301769256592), ('hall', 0.85158371925354), ('musgrove', 0.8512271642684937), ('navy', 0.8282572031021118), ('wentworth', 0.8096705079078674), ('goddard', 0.8042337894439697), ('charles', 0.8005120158195496), ('room', 0.7979660630226135)]
0.8628302
marriage
[('clay', 0.8836112022399902), ('benwick', 0.8775843381881714), ('harville', 0.8717120885848999), ('musgrove', 0.8633527159690857), ('hall', 0.8589537143707275), ('wentworth', 0.8165450692176819), ('navy', 0.8146291971206665), ('room', 0.8055905103683472), ('goddard', 0.805363655090332), ('charles', 0.802318

It Looks like it best performed when used 'hs'= 1 and 'size' = 350 however it seems that there is some level of overfitting.